In [ ]:
%load_ext autoreload
%autoreload 2   

# Node2vec

https://arxiv.org/abs/1607.00653

https://pytorch-geometric.readthedocs.io/en/latest/tutorial/shallow_node_embeddings.html

https://spotintelligence.com/2024/01/18/node2vec/

In [ ]:
import torch
import torch.nn as nn
import networkx as nx
import numpy as np
from icecream import ic

from sklearn.decomposition import PCA
import torch.optim as optim

import random

import matplotlib.pyplot as plt


device = "mps" if torch.backends.mps.is_available() else "cpu"
ic(device);

In [ ]:
# Set seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


In [ ]:
# Load the Zachary Karate Club graph
G = nx.karate_club_graph()
num_nodes = G.number_of_nodes()


def plot_graph(G, figsize=(10, 7)):
    club_dict = nx.get_node_attributes(G, "club")
    colors = [
        "lightblue" if club_dict[node] == "Mr. Hi" else "red" for node in G.nodes()
    ]
    plt.figure(figsize=figsize)
    nx.draw(
        G,
        pos=nx.spring_layout(G, seed=42),
        with_labels=True,
        node_color=colors,
        edge_color="gray",
        cmap="Set2",
    )


# plot_graph(G, figsize=(5, 5))

![](../resources/biased_rw.png)

in get_alias_edge: 
- src= t
- dst= w

In [ ]:
def get_alias_edge(G, src, curr, p, q):
    """Compute transition probabilities for neighbors of 'dst' based on Node2Vec bias.
    dst: current node
    src: previous node
    p: return back to previous node. Lower p means backtracking is more likely.
    q: in-out param: explore new nodes (ratio of bfs[breadth-first search] to dfs[depth-first search]).
        lower q means more exploration of distant nodes
    """
    unnormalized_probs = []
    for neighbor in G[curr]:
        if neighbor == src:
            weight = 1.0 / p
        elif G.has_edge(
            neighbor, src
        ):  # if neighbor is a neighbor of src (above ex: s1 is neighbor of t)
            weight = 1.0
        else:
            weight = 1.0 / q
        unnormalized_probs.append(weight)
    norm_const = sum(unnormalized_probs)
    normalized_probs = [w / norm_const for w in unnormalized_probs]
    return list(G[curr]), normalized_probs


def node2vec_walk(G, start, walk_length, p, q):
    """Generate a random walk starting from the given node."""
    walk = [start]
    while len(walk) < walk_length:
        cur = walk[-1]
        neighbors = list(G.neighbors(cur))
        if len(neighbors) == 0:
            break
        if len(walk) == 1:
            next_node = random.choice(neighbors)
        else:
            prev = walk[-2]
            candidates, probs = get_alias_edge(G, src=prev, curr=cur, p=p, q=q)
            next_node = random.choices(candidates, weights=probs, k=1)[0]
        walk.append(next_node)
    return walk


ic.enable()
nodes = list(G.nodes())
node = nodes[0]
ic(node)
for i in range(10):
    walk = node2vec_walk(G, start=node, walk_length=10, p=1, q=1)
    ic(walk)

# decompose walk

![](../resources/zachary.png)

In [ ]:
start = 0
walk = [start]
p = 1
q = 0.5
walk_length = 2
for i in range(walk_length):
    cur = walk[-1]
    neighbors = list(G.neighbors(cur))
    ic(len(neighbors))
    ic(neighbors)

    if len(neighbors) == 0:
        break
    if len(walk) == 1:
        next_node = random.choice(neighbors)
    else:
        prev = walk[-2]
        candidates, probs = get_alias_edge(G, prev, cur, p, q)
        ic(candidates)
        ic(probs)
        next_node = random.choices(candidates, weights=probs, k=1)[0]
    ic(next_node)
    walk.append(next_node)
    ic(walk)

In [ ]:
# Generate random walks for all nodes

# Node2Vec parameters
p = 1
q = 0.2
walk_length = 8
num_walks = 20
walks = []
for _ in range(num_walks):
    nodes = list(G.nodes())
    random.shuffle(nodes)
    for node in nodes:
        walk = node2vec_walk(G, node, walk_length, p, q)
        walks.append(walk)


In [ ]:
len(walks[:])

In [ ]:
window_size = 2
embedding_dim = 16
num_negative_samples = 5


# A walk represents a sequence of nodes traversed in the graph.
# For each node (called target) in a walk, the algorithm creates training pairs
# with surrounding nodes within a window of size window_size.
# window size = 2 means that for each target node,
# the algorithm will consider the two nodes before and two nodes after it in the walk.


def gen_walk_pairs(walk, window_size):
    walk_pairs = []
    for i, target in enumerate(walk):
        start = max(0, i - window_size)
        end = min(len(walk), i + window_size + 1)
        ic(i, target, start, end)
        for j in range(start, end):
            if i != j:
                ic("pair", target, j, walk[j])
                if walk[j] != target:  # Avoid pairs of the same node
                    walk_pairs.append((target, walk[j]))

    return walk_pairs


ic.enable()
w = walks[0]
ic(window_size)
w = [0, 2, 3, 2, 5, 2, 5]
ic(w)
walk_pairs = gen_walk_pairs(w, window_size)


In [ ]:
walk_pairs[:]

In [ ]:
ic.disable()
pairs = []
for walk in walks:
    walk_pairs = gen_walk_pairs(walk, window_size)
    pairs.extend(walk_pairs)

pairs = np.array(pairs)
len(pairs)


In [ ]:
from collections import Counter

# Compute pair occurrences
pair_counts = Counter(tuple(pair) for pair in pairs)
pair_counts
# Display the results
print(f"Total number of pairs: {len(pairs)}")
print(f"Number of unique pairs: {len(pair_counts)}")
print("\nTop 10 most frequent pairs:")
for pair, count in pair_counts.most_common(10):
    print(f"Pair {pair}: {count} occurrences")

# You can also create a dictionary for easy lookup
pair_dict = dict(pair_counts)
print(f"\nExample lookups:")
print(f"Pair (0, 1) occurs {pair_dict.get((0, 1), 0)} times")
print(f"Pair (1, 0) occurs {pair_dict.get((1, 0), 0)} times")

# Distribution of pair frequencies
freq_distribution = Counter(pair_counts.values())
print(f"\nFrequency distribution:")
for freq, count in sorted(freq_distribution.items()):
    print(f"{count} pairs occur {freq} time(s)")


In [ ]:
pair_dict.get((8, 9))

![](../resources/loss_rw.png)

In [ ]:
ic.enable()
ic(num_negative_samples)
batch_size = 2
negative_batch = torch.LongTensor(
    np.random.randint(0, num_nodes, size=(batch_size, num_negative_samples))
).to(device)
ic(negative_batch);

In [ ]:
# Define the Node2Vec Skip-Gram model
class Node2Vec(nn.Module):
    def __init__(self, num_nodes, embedding_dim):
        super(Node2Vec, self).__init__()
        self.in_embeddings = nn.Embedding(num_nodes, embedding_dim)
        self.out_embeddings = nn.Embedding(num_nodes, embedding_dim)
        nn.init.xavier_uniform_(self.in_embeddings.weight)
        nn.init.xavier_uniform_(self.out_embeddings.weight)

    def forward(self, target, context, negative):
        ic(target.shape, context.shape, negative.shape)
        embed_target = self.in_embeddings(target)  # (batch, embedding_dim)
        embed_context = self.out_embeddings(context)  # (batch, embedding_dim)
        ic(embed_target.shape, embed_context.shape)
        score = torch.sum(embed_target * embed_context, dim=1)
        ic(score)
        log_target = torch.log(torch.sigmoid(score) + 1e-10)
        ic(log_target)

        embed_negative = self.out_embeddings(
            negative
        )  # (batch, num_negative, embedding_dim)
        ic(embed_negative.shape)
        # we could use matmul, but bmm has been specially optimized for batch processing
        ic(torch.bmm(embed_negative, embed_target.unsqueeze(2)))
        neg_score = torch.bmm(embed_negative, embed_target.unsqueeze(2)).squeeze()
        log_negative = torch.sum(torch.log(torch.sigmoid(-neg_score) + 1e-10), dim=1)

        loss = -(log_target + log_negative)
        ic(loss)
        return loss.mean()


ic(embedding_dim, num_nodes)
model = Node2Vec(num_nodes, embedding_dim).to(device)

model.forward(
    target=torch.LongTensor([0, 1]).to(device),
    context=torch.LongTensor([1, 2]).to(device),
    negative=torch.LongTensor([[3, 2, 4], [4, 6, 5]]).to(device),
)


### Characterize the Spatial Properties of embeddings

https://medium.com/data-science-collective/whats-happening-to-embeddings-during-training-338c420705e5

To start the study, we first need some indicators or metrics that characterize the spatial properties of embeddings. Among the limited literature, the following measures were selected:

- Gini Index: Measures inequality of values, i.e., if the information in the vector is concentrated in a few dimensions.
- Vector Entropy: Measures distributional uncertainty that reveals how uniformly the embedding uses its dimensions.
- Hoyer Sparsity: Measures how many dimensions are effectively utilized.
- Spectral Entropy: Gives a frequency-domain view that reflects how smooth or noisy the embedding is.

In [ ]:
import numpy as np


def gini(x):
    """Compute the Gini coefficient of a vector."""
    x = np.abs(x.flatten()) + 1e-12  # Avoid division by zero
    x_sorted = np.sort(x)
    n = len(x)
    cumulative = np.cumsum(x_sorted)
    gini_coeff = (n + 1 - 2 * np.sum(cumulative) / cumulative[-1]) / n
    return gini_coeff


def vector_entropy(x):
    """Compute entropy of normalized absolute vector components."""
    x = np.abs(x.flatten()) + 1e-12
    p = x / np.sum(x)
    return -np.sum(p * np.log(p))


def hoyer_sparsity(x):
    """Compute Hoyer's sparsity measure of a vector."""
    x = np.abs(x.flatten())
    n = len(x)
    l1 = np.sum(x)
    l2 = np.sqrt(np.sum(x**2))
    if l1 == 0:
        return 0.0
    return (np.sqrt(n) - (l1 / l2)) / (np.sqrt(n) - 1)


def spectral_entropy(E):
    """Compute spectral entropy from an embedding matrix E (rows = vectors)."""
    # Compute covariance matrix
    cov = np.cov(E, rowvar=False)
    # Compute eigenvalues
    eigvals = np.linalg.eigvalsh(cov)
    eigvals = np.clip(eigvals, a_min=1e-12, a_max=None)  # avoid log(0)
    p = eigvals / np.sum(eigvals)
    return -np.sum(p * np.log(p))

In [ ]:
model = Node2Vec(num_nodes, embedding_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
# Training loop
num_epochs = 100
batch_size = 64
num_batches = len(pairs) // batch_size + 1
ic.disable()
for epoch in range(num_epochs):
    np.random.shuffle(pairs)
    epoch_loss = 0
    for i in range(0, len(pairs), batch_size):
        batch = pairs[i : i + batch_size]
        ic(batch.shape, batch[:, 0].shape)
        target_batch = torch.LongTensor(batch[:, 0]).to(device)
        context_batch = torch.LongTensor(batch[:, 1]).to(device)
        # Nb: there is no guarantee that the negative samples are actually negative (not connected to target nodes)
        # to be perfectly negative, For each target node, sample from nodes that are not its neighbors.
        # Not perfect by still works (just noiser.)
        negative_batch = torch.LongTensor(
            np.random.randint(0, num_nodes, size=(len(batch), num_negative_samples))
        ).to(device)

        optimizer.zero_grad()
        loss = model(target_batch, context_batch, negative_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}, Loss: {epoch_loss / num_batches:.4f}")
        embeddings = model.in_embeddings.weight.data.cpu().numpy()
        gini_score = round(float(np.mean(list(map(gini, embeddings)))), 4)
        hoyer_sparsity_score = round(
            float(np.mean(list(map(hoyer_sparsity, embeddings)))), 4
        )
        vector_entropy_score = round(
            float(np.mean(list(map(vector_entropy, embeddings)))), 4
        )
        spectral_entropy_score = round(float(spectral_entropy(embeddings)), 4)
        print(
            f"Gini: {gini_score}, Hoyer's Sparsity: {hoyer_sparsity_score}, Vector Entropy: {vector_entropy_score}, Spectral Entropy: {spectral_entropy_score}"
        )


In [ ]:
# Retrieve learned embeddings
embeddings = .weight.data.cpu().numpy()
print("\nLearned embeddings:")
for node in range(num_nodes):
    print(f"Node {node}: {embeddings[node]}")

# Use PCA to reduce the embeddings to 2D for visualization
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embeddings)

# Create colors list based on club membership: red for 'Mr. Hi', blue for others.
colors = []
for node in range(num_nodes):
    club = G.nodes[node]["club"]
    colors.append("red" if club == "Mr. Hi" else "blue")

# Plotting the PCA visualization
plt.figure(figsize=(8, 6))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=colors)
for i in range(num_nodes):
    plt.annotate(str(i), (embeddings_2d[i, 0], embeddings_2d[i, 1]), fontsize=9)
plt.title("PCA Visualization of Node2Vec Embeddings")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.show()

## Classifier on node

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
)
from sklearn.preprocessing import StandardScaler
import pandas as pd


def create_edge_features(embeddings, edge_list, method="hadamard"):
    """
    Create edge features from node embeddings using different combination methods.

    Args:
        embeddings: Node embeddings matrix (num_nodes, embedding_dim)
        edge_list: List of edges [(node1, node2), ...]
        method: Method to combine embeddings ('hadamard', 'concat', 'l1', 'l2', 'average')

    Returns:
        Feature matrix for edges
    """
    features = []

    for node1, node2 in edge_list:
        emb1 = embeddings[node1]
        emb2 = embeddings[node2]

        if method == "hadamard":
            # Element-wise multiplication (Hadamard product)
            feature = emb1 * emb2
        elif method == "concat":
            # Concatenation
            feature = np.concatenate([emb1, emb2])
        elif method == "l1":
            # L1 distance (absolute difference)
            feature = np.abs(emb1 - emb2)
        elif method == "l2":
            # L2 distance (squared difference)
            feature = (emb1 - emb2) ** 2
        elif method == "average":
            # Average of embeddings
            feature = (emb1 + emb2) / 2
        else:
            raise ValueError(f"Unknown method: {method}")

        features.append(feature)

    return np.array(features)


def prepare_link_prediction_data(G, embeddings, test_ratio=0.2, neg_pos_ratio=1.0):
    """
    Prepare data for link prediction task.

    Args:
        G: NetworkX graph
        embeddings: Node embeddings
        test_ratio: Ratio of edges to use for testing
        neg_pos_ratio: Ratio of negative to positive samples

    Returns:
        Train and test data with features and labels
    """
    # Get all edges (positive examples)
    edges = list(G.edges())

    # Split edges into train and test
    train_edges, test_edges = train_test_split(
        edges, test_size=test_ratio, random_state=42
    )

    # Generate negative examples (non-existing edges)
    def generate_negative_edges(G, num_neg_edges, existing_edges_set):
        neg_edges = []
        nodes = list(G.nodes())

        while len(neg_edges) < num_neg_edges:
            node1 = random.choice(nodes)
            node2 = random.choice(nodes)

            if (
                node1 != node2
                and (node1, node2) not in existing_edges_set
                and (node2, node1) not in existing_edges_set
            ):
                neg_edges.append((node1, node2))

        return neg_edges

    # Create sets of existing edges for efficient lookup
    all_edges_set = set(edges + [(v, u) for u, v in edges])  # Include both directions

    # Generate negative examples
    num_train_neg = int(len(train_edges) * neg_pos_ratio)
    num_test_neg = int(len(test_edges) * neg_pos_ratio)

    train_neg_edges = generate_negative_edges(G, num_train_neg, all_edges_set)
    test_neg_edges = generate_negative_edges(G, num_test_neg, all_edges_set)

    return {
        "train_pos": train_edges,
        "train_neg": train_neg_edges,
        "test_pos": test_edges,
        "test_neg": test_neg_edges,
    }


In [ ]:
data = prepare_link_prediction_data(G, embeddings, 0.1)


In [ ]:
# Prepare data
test_ratio = 0.1
data = prepare_link_prediction_data(G, embeddings, test_ratio)
ic(data.keys())

method = "hadamard"
method = "concat"
# Create features for training data
train_pos_features = create_edge_features(embeddings, data["train_pos"], method)
train_neg_features = create_edge_features(embeddings, data["train_neg"], method)

# Create features for test data
test_pos_features = create_edge_features(embeddings, data["test_pos"], method)
test_neg_features = create_edge_features(embeddings, data["test_neg"], method)

In [ ]:
train_pos_features.shape

In [ ]:
# Combine positive and negative samples
X_train = np.vstack([train_pos_features, train_neg_features])
y_train = np.hstack(
    [np.ones(len(train_pos_features)), np.zeros(len(train_neg_features))]
)
y_train, X_train

In [ ]:
results = []

# Combine positive and negative samples
X_train = np.vstack([train_pos_features, train_neg_features])
y_train = np.hstack(
    [np.ones(len(train_pos_features)), np.zeros(len(train_neg_features))]
)

X_test = np.vstack([test_pos_features, test_neg_features])
y_test = np.hstack([np.ones(len(test_pos_features)), np.zeros(len(test_neg_features))])

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
X_train_final = X_train  # Random Forest doesn't need scaling
X_test_final = X_test

# Train classifier
clf.fit(X_train_final, y_train)

# Make predictions
y_pred = clf.predict(X_test_final)
y_pred_proba = clf.predict_proba(X_test_final)[:, 1]

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"AUC-ROC: {auc:.4f}")

results.append(
    {
        "method": method,
        "classifier": "random_forest",
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "auc": auc,
    }
)


# Convert results to DataFrame for better visualization
results_df = pd.DataFrame(results)
print("\n=== Summary Results ===")
print(results_df.round(4))

In [ ]:
def predict_node_relationship(
    node1, node2, embeddings, classifier, scaler, method="hadamard"
):
    """
    Predict if two specific nodes have a relationship.

    Args:
        node1, node2: Node indices
        embeddings: Node embeddings matrix
        classifier: Trained classifier
        scaler: Fitted scaler
        method: Feature combination method

    Returns:
        Prediction probability and binary prediction
    """
    # Create edge feature
    edge_feature = create_edge_features(embeddings, [(node1, node2)], method)

    # Scale the feature if using logistic regression
    if hasattr(classifier, "predict_proba"):
        try:
            edge_feature_scaled = scaler.transform(edge_feature)
            prediction_proba = classifier.predict_proba(edge_feature_scaled)[0, 1]
            prediction = classifier.predict(edge_feature_scaled)[0]
        except:
            # For Random Forest, use unscaled features
            prediction_proba = classifier.predict_proba(edge_feature)[0, 1]
            prediction = classifier.predict(edge_feature)[0]
    else:
        prediction_proba = classifier.predict_proba(edge_feature)[0, 1]
        prediction = classifier.predict(edge_feature)[0]

    return prediction_proba, prediction


def analyze_node_relationships(G, embeddings, classifier, scaler, method="hadamard"):
    """
    Analyze relationships for all node pairs and compare with actual graph structure.
    """
    nodes = list(G.nodes())
    predictions = []

    print("Analyzing node relationships...")
    print("Format: (node1, node2) -> probability, prediction, actual")
    print("-" * 60)

    # Check some interesting pairs
    interesting_pairs = [
        (0, 33),  # The two main leaders
        (0, 1),  # Direct neighbors
        (1, 2),  # Another direct connection
        (5, 6),  # Members of same club
        (15, 20),  # Members of different clubs
        (10, 15),  # Random pair
    ]

    for node1, node2 in interesting_pairs:
        prob, pred = predict_node_relationship(
            node1, node2, embeddings, classifier, scaler, method
        )
        actual = 1 if G.has_edge(node1, node2) else 0

        print(
            f"({node1:2d}, {node2:2d}) -> {prob:.4f}, {pred}, {actual} {'✓' if pred == actual else '✗'}"
        )
        predictions.append(
            {
                "node1": node1,
                "node2": node2,
                "probability": prob,
                "prediction": pred,
                "actual": actual,
                "correct": pred == actual,
            }
        )

    return predictions


# Test predictions on specific node pairs
print("\\n=== Predicting Specific Node Relationships ===")
predictions = analyze_node_relationships(G, embeddings, trained_clf, scaler)


# Create a heatmap of prediction probabilities
def create_prediction_heatmap(
    G, embeddings, classifier, scaler, method="hadamard", sample_size=None
):
    """
    Create a heatmap showing prediction probabilities for node pairs.
    """
    nodes = list(G.nodes())
    if sample_size and len(nodes) > sample_size:
        nodes = random.sample(nodes, sample_size)

    n_nodes = len(nodes)
    prob_matrix = np.zeros((n_nodes, n_nodes))

    for i, node1 in enumerate(nodes):
        for j, node2 in enumerate(nodes):
            if i != j:
                prob, _ = predict_node_relationship(
                    node1, node2, embeddings, classifier, scaler, method
                )
                prob_matrix[i, j] = prob

    plt.figure(figsize=(10, 8))
    plt.imshow(prob_matrix, cmap="RdYlBu_r", vmin=0, vmax=1)
    plt.colorbar(label="Relationship Probability")
    plt.title("Predicted Relationship Probabilities Between Nodes")
    plt.xlabel("Node Index")
    plt.ylabel("Node Index")

    # Add tick labels
    plt.xticks(range(n_nodes), nodes)
    plt.yticks(range(n_nodes), nodes)

    plt.tight_layout()
    plt.show()

    return prob_matrix


# Create prediction heatmap for first 15 nodes
print("\\n=== Creating Prediction Heatmap ===")
prob_matrix = create_prediction_heatmap(
    G, embeddings, trained_clf, scaler, sample_size=15
)

# Feature importance analysis (for Random Forest)
if hasattr(trained_clf, "feature_importances_"):
    print("\\n=== Feature Importance Analysis ===")
    feature_importance = trained_clf.feature_importances_

    plt.figure(figsize=(12, 6))
    plt.bar(range(len(feature_importance)), feature_importance)
    plt.title("Feature Importance for Link Prediction")
    plt.xlabel("Feature Index")
    plt.ylabel("Importance")
    plt.tight_layout()
    plt.show()

    print(f"Top 5 most important features: {np.argsort(feature_importance)[-5:][::-1]}")
    print(
        f"Their importance scores: {feature_importance[np.argsort(feature_importance)[-5:][::-1]]}"
    )

## gemini

Solution suggested by gemini

In [ ]:
import torch
import networkx as nx
import matplotlib.pyplot as plt
import random
from collections import defaultdict
from tqdm import tqdm


# Load the Zachary Karate Club graph
def load_karate_graph():
    """
    Loads the Zachary Karate Club graph from NetworkX.

    Returns:
        nx.Graph: The Karate Club graph.
    """
    G = nx.karate_club_graph()
    return G


# Convert NetworkX graph to adjacency list
def graph_to_adj_list(graph):
    """
    Converts a NetworkX graph to an adjacency list representation.

    Args:
        graph (nx.Graph): The input graph.

    Returns:
        dict: An adjacency list where keys are nodes and values are lists of neighbors.
    """
    adj_list = defaultdict(list)
    for node in graph.nodes():
        adj_list[node] = list(graph.neighbors(node))
    return adj_list


# Generate random walk
def generate_random_walk(graph, start_node, walk_length, p, q):
    """
    Generates a random walk starting from a given node.  This implementation
    correctly handles the edge weights (p and q) for the walk.

    Args:
        graph (nx.Graph): The input graph.
        start_node (int): The starting node for the random walk.
        walk_length (int): The length of the random walk.
        p (float): The return parameter.
        q (float): The in-out parameter.

    Returns:
        list: A list of nodes representing the random walk.
    """
    walk = [start_node]
    for _ in range(walk_length - 1):
        current_node = walk[-1]
        neighbors = list(graph.neighbors(current_node))
        if not neighbors:
            break  # Handle disconnected nodes
        if len(walk) == 1:
            # Start node case: sample directly from neighbors
            next_node = random.choice(neighbors)
            walk.append(next_node)
        else:
            # Subsequent step case: consider p and q
            previous_node = walk[-2]
            probabilities = []
            for neighbor in neighbors:
                if neighbor == previous_node:
                    probabilities.append(1 / p)  # Return probability
                elif graph.has_edge(previous_node, neighbor):
                    probabilities.append(1)  # Neighboring node
                else:
                    probabilities.append(1 / q)  # Non-neighboring node
            # Normalize probabilities to sum to 1
            probabilities = [prob / sum(probabilities) for prob in probabilities]
            # Use the probabilities to select the next node.
            next_node = random.choices(neighbors, probabilities)[0]
            walk.append(next_node)
    return walk


# Generate multiple random walks
def generate_walks(graph, num_walks, walk_length, p, q):
    """
    Generates multiple random walks for each node in the graph.

    Args:
        graph (nx.Graph): The input graph.
        num_walks (int): The number of random walks per node.
        walk_length (int): The length of each random walk.
        p (float): The return parameter.
        q (float): The in-out parameter.

    Returns:
        list: A list of lists, where each inner list is a random walk.
    """
    walks = []
    nodes = list(graph.nodes())
    for _ in range(num_walks):
        random.shuffle(nodes)  # Shuffle for each iteration
        for node in nodes:
            walk = generate_random_walk(graph, node, walk_length, p, q)
            walks.append(walk)
    return walks


# Skip-gram model with negative sampling
class SkipGramModel(torch.nn.Module):
    """
    Skip-gram model for learning node embeddings.  This implementation
    uses PyTorch and includes proper initialization.
    """

    def __init__(self, num_nodes, embedding_dim):
        """
        Initializes the Skip-gram model.

        Args:
            num_nodes (int): The number of nodes in the graph.
            embedding_dim (int): The dimensionality of the node embeddings.
        """
        super(SkipGramModel, self).__init__()
        self.embedding_u = torch.nn.Embedding(
            num_nodes, embedding_dim
        )  # Target embeddings
        self.embedding_v = torch.nn.Embedding(
            num_nodes, embedding_dim
        )  # Context embeddings

        # Initialize embeddings using a small uniform distribution
        init_range = 0.5 / embedding_dim
        self.embedding_u.weight.data.uniform_(-init_range, init_range)
        self.embedding_v.weight.data.uniform_(
            -0, 0
        )  # init_range, init_range) # Initializing to 0 can sometimes help

    def forward(self, center_nodes, context_nodes, negative_samples):
        """
        Computes the forward pass of the Skip-gram model.

        Args:
            center_nodes (torch.Tensor): Tensor of center node indices.
            context_nodes (torch.Tensor): Tensor of context node indices.
            negative_samples (torch.Tensor): Tensor of negative sample indices.

        Returns:
            torch.Tensor: The loss value.
        """
        # Get embeddings for center nodes
        u_embeddings = self.embedding_u(center_nodes)  # (batch_size, embedding_dim)

        # Get embeddings for context nodes
        v_embeddings = self.embedding_v(context_nodes)  # (batch_size, embedding_dim)

        # Compute the positive log-likelihood
        positive_scores = torch.sum(u_embeddings * v_embeddings, dim=1)  # (batch_size,)
        positive_loss = -torch.mean(torch.log(torch.sigmoid(positive_scores)))

        # Get embeddings for negative samples
        negative_embeddings = self.embedding_v(
            negative_samples
        )  # (batch_size, num_negative_samples, embedding_dim)

        # Compute the negative log-likelihood
        negative_scores = torch.bmm(
            negative_embeddings, u_embeddings.unsqueeze(2)
        ).squeeze(2)  # (batch_size, num_negative_samples)
        negative_loss = -torch.mean(torch.log(torch.sigmoid(-negative_scores)))

        # Combine positive and negative losses
        loss = positive_loss + negative_loss
        return loss


def generate_negative_samples(walks, num_nodes, window_size, num_negative_samples):
    """
    Generates negative samples for each center node and context node pair.
    This version pre-computes the unigram distribution and samples directly
    according to that distribution.

    Args:
        walks (list): A list of random walks.
        num_nodes (int): The total number of nodes in the graph.
        window_size (int): The context window size.
        num_negative_samples (int): The number of negative samples per (center, context) pair.

    Returns:
        list: A list of tuples, where each tuple contains:
            (center_node, context_node, list of negative samples)
    """
    negative_samples = []
    node_counts = [0] * num_nodes  # Initialize node counts
    for walk in walks:
        for node in walk:
            node_counts[node] += 1

    # Compute the unigram distribution raised to the power of 0.75
    total_count = sum(node_counts)
    unigram_dist = [(count / total_count) ** 0.75 for count in node_counts]
    # Normalize the distribution
    unigram_dist = [p / sum(unigram_dist) for p in unigram_dist]

    for walk in walks:
        for i, center_node in enumerate(walk):
            for j in range(
                max(0, i - window_size), min(len(walk), i + window_size + 1)
            ):
                if i != j:
                    context_node = walk[j]
                    # Generate negative samples, ensuring they are different from the context node
                    neg_samples = random.choices(
                        range(num_nodes), weights=unigram_dist, k=num_negative_samples
                    )
                    negative_samples.append((center_node, context_node, neg_samples))
    return negative_samples


def train_node2vec(
    graph,
    walks,
    embedding_dim,
    window_size,
    num_negative_samples,
    batch_size,
    epochs,
    learning_rate,
    p,
    q,
    num_workers=0,
):
    """
    Trains the Node2vec model. This function incorporates the negative sampling
    and training loop, and uses the graph, walks, and p, q parameters.

    Args:
        graph (nx.Graph): The input graph.
        walks (list): A list of random walks.
        embedding_dim (int): The dimensionality of the node embeddings.
        window_size (int): The context window size.
        num_negative_samples (int): The number of negative samples per (center, context) pair.
        batch_size (int): The batch size for training.
        epochs (int): The number of training epochs.
        learning_rate (float): The learning rate.
        p (float): The return parameter.
        q (float): The in-out parameter.
        num_workers (int): Number of workers for data loading (not used in this basic implementation).

    Returns:
        torch.nn.Module: The trained Skip-gram model.
    """
    # Determine the device to use
    device = (
        torch.device("mps")
        if torch.backends.mps.is_available()
        else torch.device("cpu")
    )
    print(f"Using device: {device}")

    num_nodes = graph.number_of_nodes()
    model = SkipGramModel(num_nodes, embedding_dim).to(device)  # Move model to device
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    # Use Binary Cross Entropy Loss (though we calculate it manually for more control)
    # criterion = torch.nn.BCEWithLogitsLoss() # Not used, but kept here for reference

    negative_samples = generate_negative_samples(
        walks, num_nodes, window_size, num_negative_samples
    )

    # Prepare data for PyTorch (create tensors) outside the loop
    center_nodes_all = []
    context_nodes_all = []
    negative_samples_all = []

    for center_node, context_node, neg_samples in negative_samples:
        center_nodes_all.append(center_node)
        context_nodes_all.append(context_node)
        negative_samples_all.append(neg_samples)

    center_nodes_tensor = torch.tensor(center_nodes_all, dtype=torch.long).to(
        device
    )  # Move tensors to device
    context_nodes_tensor = torch.tensor(context_nodes_all, dtype=torch.long).to(device)
    negative_samples_tensor = torch.tensor(negative_samples_all, dtype=torch.long).to(
        device
    )

    num_samples = len(negative_samples)
    for epoch in range(epochs):
        # Shuffle the training data at the beginning of each epoch
        permutation = torch.randperm(num_samples).to(
            device
        )  # Move permutation to device
        center_nodes_tensor = center_nodes_tensor[permutation]
        context_nodes_tensor = context_nodes_tensor[permutation]
        negative_samples_tensor = negative_samples_tensor[permutation]
        total_loss = 0.0
        for i in tqdm(
            range(0, num_samples, batch_size), desc=f"Epoch {epoch + 1}/{epochs}"
        ):
            optimizer.zero_grad()
            # Get batch
            indices = torch.arange(i, min(i + batch_size, num_samples)).to(
                device
            )  # Move indices to device
            batch_center_nodes = center_nodes_tensor[indices]
            batch_context_nodes = context_nodes_tensor[indices]
            batch_negative_samples = negative_samples_tensor[indices]

            # Forward pass
            loss = model(
                batch_center_nodes, batch_context_nodes, batch_negative_samples
            )
            total_loss += loss.item()
            # Backward pass
            loss.backward()
            optimizer.step()
        print(
            f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / (num_samples // batch_size)}"
        )
    return model


def get_node_embeddings(model):
    """
    Gets the node embeddings from the trained model.

    Args:
        model (torch.nn.Module): The trained Skip-gram model.

    Returns:
        torch.Tensor: A tensor containing the node embeddings.
    """
    return model.embedding_u.weight.data.cpu().numpy()


def visualize_embeddings(embeddings, graph, title="Node2vec Embeddings"):
    """
    Visualizes the node embeddings using a 2D scatter plot.  This version
    uses the actual node labels from the Karate Club graph.

    Args:
        embeddings (numpy.ndarray): The node embeddings.
        graph (nx.Graph): The input graph (for getting node labels).
        title (str): The title of the plot.
    """
    # Use a simple PCA for dimensionality reduction to 2D
    from sklearn.decomposition import PCA

    pca = PCA(n_components=2)
    reduced_embeddings = pca.fit_transform(embeddings)

    # Get the club labels for coloring
    club_labels = [graph.nodes[node]["club"] for node in graph.nodes()]
    colors = ["red" if label == "Mr. Hi" else "blue" for label in club_labels]

    plt.figure(figsize=(10, 8))
    plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], c=colors)

    # Add node labels to the plot
    for i, node in enumerate(graph.nodes()):
        plt.annotate(
            str(node),
            xy=(reduced_embeddings[i, 0], reduced_embeddings[i, 1]),
            xytext=(5, 2),
            textcoords="offset points",
            fontsize=10,
        )

    plt.title(title)
    plt.xlabel("Dimension 1")
    plt.ylabel("Dimension 2")
    plt.grid(True)
    plt.show()


if __name__ == "__main__":
    # Hyperparameters
    embedding_dim = 128
    walk_length = 20
    num_walks = 10
    window_size = 5
    num_negative_samples = 5
    batch_size = 32
    epochs = 50
    learning_rate = 0.05  # Increased learning rate
    p = 1.0
    q = 1.0

    # Load graph
    karate_graph = load_karate_graph()

    # Generate random walks
    walks = generate_walks(karate_graph, num_walks, walk_length, p, q)

    # Train Node2vec model
    model = train_node2vec(
        karate_graph,
        walks,
        embedding_dim,
        window_size,
        num_negative_samples,
        batch_size,
        epochs,
        learning_rate,
        p,
        q,
    )

    # Get node embeddings
    node_embeddings = get_node_embeddings(model)

    # Visualize embeddings
    visualize_embeddings(
        node_embeddings, karate_graph, title="Node2vec Embeddings (Karate Club)"
    )

    # Print the first 5 embeddings
    print("First 5 Node Embeddings:")
    print(node_embeddings[:5])
